In [2]:
#from IPython.core.display import display, HTML
#display(HTML("<style>div.output_scroll { height: 44em; }</style>"))

from zipfile import ZipFile
import uproot
import matplotlib.pyplot as plt
import pandas as pd
import math
import numpy as np
import os,glob,time
import csv
import subprocess
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy.signal import find_peaks
import scipy.optimize as opt
import matplotlib.mlab as mlab
from scipy.optimize import leastsq
from sklearn import preprocessing
import peakutils

from astropy.stats import sigma_clipped_stats
from photutils.datasets import make_100gaussians_image
#from photutils import find_peaks
from astropy.visualization import simple_norm
from astropy.visualization.mpl_normalize import ImageNormalize
from photutils import CircularAperture
import matplotlib.ticker as ticker
from matplotlib.ticker import AutoMinorLocator, MultipleLocator, FuncFormatter

import memory_profiler

from IPython.display import Markdown as md


import re
numeric_const_pattern = '[-+]? (?: (?: \d* \. \d+ ) | (?: \d+ \.? ) )(?: [Ee] [+-]? \d+ ) ?'
rx = re.compile(numeric_const_pattern, re.VERBOSE)


plt.style.use('seaborn-white')

plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 22
plt.rcParams['axes.labelsize'] = 22
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 22
plt.rcParams['xtick.labelsize'] = 22
plt.rcParams['ytick.labelsize'] = 22
plt.rcParams['legend.fontsize'] = 18
plt.rcParams['figure.titlesize'] = 22


In [3]:
def MonNum(Mon):
    Mon_Strs = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
    for i,m in enumerate(Mon_Strs):
        if Mon == m:
            return i+1
            
    return -1

In [4]:
def file_info_from_file(file):
    #fdata=file.find("data")
    #file=file[fdata:]
    first_dash=file.find("-")
    last_slash=file.rfind("/")
    Config=file[last_slash+1:first_dash]
    file=file[first_dash+1:]
    version="0"
    if file.find("v")>=0:
        version_dash=file.find("-")
        version=file[:version_dash]
        file=file[version_dash+1:]
    else:
        verison="0"
    
    tgt_dash=file.find("-")
    tgt=file[:tgt_dash]
    file=file[tgt_dash+1:]
    
    evts_dash=file.find("-")
    numevents=int(file[:evts_dash])
    seed=file[file.find("seed")+5:file.find(".")]
    return Config,tgt,numevents,seed,version

def file_dict_from_file(file):
    
    fileinfo=file_info_from_file(file)
    simFiledict={}
    simFiledict["Config"]=fileinfo[0]
    simFiledict["Target"]=fileinfo[1]
    simFiledict["NumEvents"]=fileinfo[2]
    simFiledict["Seed"]=fileinfo[3]
    simFiledict["Version"]=fileinfo[4]
    return simFiledict


def haveit(DF):

    paths=["/home/wrkshp/nEXO/slurm_data/"+DF["Iso"]+"/Unzipped"
           ,"/home/wrkshp/nEXO/slurm_data/"+DF["Iso"]
           ,"/home/wrkshp/nEXO/slurm_data"]

    for i,path in enumerate(paths):
        file_name=path+"/"+DF["Name"]
        if os.path.exists(file_name):
            return file_name,i
    return -1,-1


#def slac_move_old(DB):

    
    
def getit(DB):
    file=DB["Name"]
    path="exo_data/data/slurm_data"
    newdir='/home/wrkshp/nexo/slurm_data/%s'%(DB["Iso"])
    print("sftp jbane@ftp.slac.stanford.edu <<< $'get %s/%s %s'"%(path,file,newdir))
    os.system( "sftp jbane@ftp.slac.stanford.edu <<< $'get %s/%s %s' >> sftp_info.txt"%(path,file,newdir))

def unzipAndmove(DB):

    file=DB["Name"]
    path='/home/wrkshp/nexo/slurm_data/%s'%(DB["Iso"])
    os.system("tar -xzf %s/%s -C %s"%(path,file,path))
    os.system("mv -v %s/%s %s/Unzipped"%(path,file,path))
    os.system("mv -v %s/*recon* /home/wrkshp/nexo/slurm_data/recon"%(path))
    
def unzipAndmove_recon(DB):

    file=DB["Name"]
    path='/home/wrkshp/nexo/slurm_data/%s'%(DB["Iso"])
    os.system("tar -xzvf %s/%s -C %s --wildcards --no-anchored *recon.root"%(path,file,path))
    os.system("mv -v %s/%s %s/Unzipped"%(path,file,path))
    os.system("mv -v %s/*recon* /home/wrkshp/nexo/slurm_data/recon"%(path))
    
    
    
    
    
def getfromnfs(DB):
    
    data_dir="/home/wrkshp/slac_storage/sshfs/nfs/slac/g/exo_data8/exo_data/users/jbane/data/slurm_data"
    new_dir ="/home/wrkshp/nexo/slurm_data/%s"%(DB["Iso"])
    file=DB["Name"]
    if os.path.exists("%s/%s"%(data_dir,file)):
        a=subprocess.check_output("cp -v %s/%s %s"%(data_dir,file,new_dir), shell=True)
    else:
        print("Not in nfs")
        a=False
    return a

def redo(DB):
    file=DB["Name"]
    path='/home/wrkshp/nexo/slurm_data/%s'%(DB["Iso"])
    try:
        a=subprocess.check_output("tar -xzf %s/Unzipped/%s -C %s"%(path,file,path), shell=True)
        
    except:
        print("Issue with tar file")
        return 0
    os.system("mv -v %s/*recon* /home/wrkshp/nexo/slurm_data/recon"%(path))    
    return 1
    #os.system("mv -v %s/%s %s/Unzipped"%(path,file,path))

def redo_recon(DB):
    file=DB["Name"]
    path='/home/wrkshp/nexo/slurm_data/%s'%(DB["Iso"])
    try:
        a=subprocess.check_output("tar -xzf %s/Unzipped/%s -C %s --wildcards --no-anchored *recon.root"%(path,file,path), shell=True)
        
    except:
        print("Issue with tar file")
        return 0
    os.system("mv -v %s/*recon* /home/wrkshp/nexo/slurm_data/recon"%(path))    
    return 1
    #os.system("mv -v %s/%s %s/Unzipped"%(path,file,path))

def redo_root(DB):
    file=DB["Name"]
    path='/home/wrkshp/nexo/slurm_data/%s'%(DB["Iso"])
    try:
        a=subprocess.check_output("tar -xzf %s/Unzipped/%s -C %s --wildcards --no-anchored *.root"%(path,file,path), shell=True)
        
    except:
        print("Issue with tar file")
        return 0
    os.system("mv -v %s/*recon* /home/wrkshp/nexo/slurm_data/recon"%(path))    
    return 1
    #os.system("mv -v %s/%s %s/Unzipped"%(path,file,path))
    
    

def checknfs():
    a=os.path.exists("/home/wrkshp/slac_storage/sshfs/nfs/slac")
    print("nfs mount is there?",a)
    if a ==False:
        os.system("sshfs jbane@dtn01.slac.stanford.edu:/nfs /home/wrkshp/slac_storage/sshfs/nfs")
def unmount_nfs():
    os.system("/bin/fusermount -u /home/wrkshp/slac_storage/sshfs/nfs")
    

In [5]:
from datetime import datetime

currentSecond= datetime.now().second
currentMinute = datetime.now().minute
currentHour = datetime.now().hour

currentDay = datetime.now().day
currentMonth = datetime.now().month
currentYear = datetime.now().year

In [6]:
stream=os.system('sh tryssh.sh > slurm_files.txt')

if stream >0:
    print("Error with ssh script")
    os.system("kinit -R jbane@SLAC.STANFORD.EDU || kinit -r 7d jbane@SLAC.STANFORD.EDU")
    stream=os.system('sh tryssh.sh > slurm_files.txt')
    if stream >0:
        print("Could not get list of tar files: need assistance")
    else:
        print("Worked on second try")
else :
    print("Looks Good")




Looks Good


In [7]:
G_mon=11
G_day=14
CurMonNum=currentMonth


print(CurMonNum)
file=open("slurm_files.txt")
lines=file.readlines()
file_dicts={}
for i,line in enumerate(lines):
    file_dict={}
    if line.find(".tar") != -1 and line.find("seed") != -1 :
        #print(line)
        file_info =line.split()
        file_dict["Mon"]=MonNum(file_info[5])
        file_dict["Date"]=int(file_info[6])
        file_dict["Year"]= 2022 if  file_dict["Mon"] <= CurMonNum else 2021
        
        #print(file_dict["Year"])
        
        file_dict["Time"]=file_info[7]
        
        file_dict["Name"]=file_info[8]
        #print(type(file_info[8]))
        mo_info=file_info_from_file( file_dict["Name"] )
        file_dict["Config"]=mo_info[0]
        file_dict["Iso"]=mo_info[1]
        file_dict["MaxEvents"]=mo_info[2]
        file_dict["Seeds"]=mo_info[3]
        file_dict["Version"]=mo_info[4]
        
        #print(file_dict)
        file_dicts[i]=file_dict
        
        
        
file_db = pd.DataFrame.from_dict(file_dicts)
file_db=file_db.transpose()
file_db= file_db[["Mon","Date","Year","Time","Config","Version","Iso","Seeds","MaxEvents","Name"]]

#file_db=file_db.query("Mon > %i and Date ")




12


In [8]:
len(file_db)


file_db[-17:]

,Mon,Date,Year,Time,Config,Version,Iso,Seeds,MaxEvents,Name
336,12,1,2022,08:27,ExternalGammas_light,v3,Bi212,47-49,5000,ExternalGammas_light-v3-Bi212-5000-seed_47-49....
337,12,1,2022,08:32,ExternalGammas_light,v3,Pb212,76-100,5000,ExternalGammas_light-v3-Pb212-5000-seed_76-100...
338,12,1,2022,08:40,ExternalGammas_light,v3,Pb212,101-125,5000,ExternalGammas_light-v3-Pb212-5000-seed_101-12...
339,12,1,2022,08:49,ExternalGammas_light,v3,Pb212,126-150,5000,ExternalGammas_light-v3-Pb212-5000-seed_126-15...
340,12,1,2022,08:57,ExternalGammas_light,v3,Pb212,151-175,5000,ExternalGammas_light-v3-Pb212-5000-seed_151-17...
341,12,1,2022,09:06,ExternalGammas_light,v3,Pb212,176-200,5000,ExternalGammas_light-v3-Pb212-5000-seed_176-20...
342,12,1,2022,10:25,ExternalGammas_light,v3,Pb212,1-25,5000,ExternalGammas_light-v3-Pb212-5000-seed_1-25.t...
343,12,1,2022,10:26,ExternalGammas_light,v3,Pb212,26-50,5000,ExternalGammas_light-v3-Pb212-5000-seed_26-50....
345,12,2,2022,06:36,ExternalGammas_light,v3,Ra224,1-25,5000,ExternalGammas_light-v3-Ra224-5000-seed_1-25.t...
346,12,2,2022,06:45,ExternalGammas_light,v3,Ra224,26-50,5000,ExternalGammas_light-v3-Ra224-5000-seed_26-50....


In [9]:
def doublecheck(DB):
    seeds=DB["Seeds"]
    first=int(seeds[:seeds.find("-")])
    last=int(seeds[seeds.find("-")+1:])
    path='/home/wrkshp/nexo/slurm_data/%s'%(DB["Iso"])
    print()
    first_check=os.path.exists("%s/%s-%s-%s-seed%i.root"%(path,DB["Config"],DB["Iso"],DB["MaxEvents"],first))
    last_check=os.path.exists("%s/%s-%s-%s-seed%i.root"%(path,DB["Config"],DB["Iso"],DB["MaxEvents"],last))
    pathrecon='/home/wrkshp/nexo/slurm_data/recon'
    recfirst_check=os.path.exists("%s/%s-%s-%s-seed%i_recon.root"%(pathrecon,DB["Config"],DB["Iso"],DB["MaxEvents"],first))
    reclast_check=os.path.exists("%s/%s-%s-%s-seed%i_recon.root"%(pathrecon,DB["Config"],DB["Iso"],DB["MaxEvents"],last))
    
    DC= first_check and last_check and recfirst_check and reclast_check
    
    print(seeds,DC)
    return(DC)



def doublecheck_recon(DB):
    seeds=DB["Seeds"]
    first=int(seeds[:seeds.find("-")])
    last=int(seeds[seeds.find("-")+1:])
    path='/home/wrkshp/nexo/slurm_data/%s'%(DB["Iso"])
    print()
    #first_check=os.path.exists("%s/%s-%s-%s-seed%i_recon.root"%(path,DB["Config"],DB["Iso"],DB["MaxEvents"],first))
    #last_check=os.path.exists("%s/%s-%s-%s-seed%i_recon.root"%(path,DB["Config"],DB["Iso"],DB["MaxEvents"],last))
    pathrecon='/home/wrkshp/nexo/slurm_data/recon'
    recfirst_check=os.path.exists("%s/%s-%s-%s-seed%i_sensrecon.root"%(pathrecon,DB["Config"],DB["Iso"],DB["MaxEvents"],first))
    reclast_check=os.path.exists("%s/%s-%s-%s-seed%i_sensrecon.root"%(pathrecon,DB["Config"],DB["Iso"],DB["MaxEvents"],last))
    
    DC= recfirst_check and reclast_check
    
    print(seeds,DC)
    return(DC)


    

In [15]:
DB=file_db[-2:]
#print(DB)
Gmon=CurMonNum
print(Gmon)
Gday=12
Gyear=2022
redo=1
Gver="v4"
reroot=1


#Ra226_list=["Ra226","Bi214","Pb214","Bi210", "Pb206"]
Th228_list=["Th228","Tl208","Ra224","Pb212","Bi212"]

#tgt_list=Th222_list
tgt_list=["Tl208","Cs137","Co60","Bi212"]
#tgt_list=["Ra224"]

for entry in DB.index:
    #if info["Iso"] not in Th228_list: #"Co60":# 
    #    print("\t\tcheck2")
    #    continue
      

    info = DB.loc[entry]
    A=haveit(info)
    
    #print(info["Iso"])
    print(info["Name"],info["Mon"],info["Date"],info["Year"],A[1],info["Iso"] not in Th228_list)
    
    if info["Year"] < 2022:
        continue
    if info["Iso"] not in tgt_list: #"Co60":# 
        continue
    if info["Version"] != Gver:
        continue
    
    
    print(info["Mon"] < Gmon, info["Mon"] == Gmon ,info["Date"] <Gday)
    
    if info["Mon"] < Gmon or (info["Mon"] == Gmon and info["Date"] <Gday):
        print("\t\tcheck3")
        continue
    
    print("\t",info["Mon"] > Gmon or (info["Mon"] == Gmon and info["Date"] >=Gday))
    
    if A[1]==-1:print("We do not have")
    elif A[1]==0:print("It has been unzipped")
    elif A[1]==1:print("It is in the right place")
    elif A[1]==2:print("Need to move it")
    else: print("issue with have it function")
    if A[1] !=-1:  #We have it
        #os.system('sh moveold.sh %s >> Movenotes.txt'%(info["Name"]) )
        if A[1]==1: #we have it in the isotope dir
            if info["Mon"] > Gmon or (info["Mon"] == Gmon and info["Date"] >=Gday):
                #unzipAndmove_recon(info)
                unzipAndmove(info)
                aaaaa=""                
            else : #in isotope dir
                path='/home/wrkshp/nexo/slurm_data/%s'%(info["Iso"])
                os.system("mv %s/%s %s/Unzipped"%(path,info["Name"],path))
        elif A[1]==2: # in slurmdata dir
            os.system("mv /home/wrkshp/nexo/slurm_data/%s /home/wrkshp/nexo/slurm_data/%s/"%(DB["Name"],DB["Iso"]))
            #unzipAndmove_recon(info)
            unzipAndmove(info)
            aaaa=""
        elif A[1] == 0: #its in slurm
            if doublecheck_recon(info) == False or redo ==1: #
                print("redoing this file")
                #os.system("mv -v info[]")
                #redo_recon(info)
                redo_root(info)
                doublecheck_recon(info)
            print()
        aa=1
    elif A[1] == -1:
        print("test",info["Mon"] >= Gmon)
        if info["Mon"] > Gmon:
            print("Grabbing file from nfs to unzip")
            
            #getit(info)
            checknfs()            
            getfromnfs(info)
            #unzipAndmove_recon(info)
            unzipAndmove(info)
                
        if info["Mon"]== Gmon:
            if info["Date"] >=Gday:
                print("Grabbing file from nfs to unzip")
            
                #getit(info)
                checknfs()
            
            
            getfromnfs(info)
            unzipAndmove(info)
            #unzipAndmove_recon(info)
        if haveit(info)[1] == -1:
            print("!!!!issue with sftp!!!!")
                #break
            #unzipAndmove(info)

unmount_nfs()
        
                

12
ExternalGammas_light-v3-Ra224-5000-seed_176-200.tar.gz 12 2 2022 0 False
ExternalGammas_light-v4-Tl208-5000-seed_1-20.tar.gz 12 14 2022 -1 False
False True False
	 True
We do not have
test True
Grabbing file from nfs to unzip
nfs mount is there? False


In [ ]:
DB=file_db.loc[168]
print(DB)
print(type(DB["Date"]))


In [ ]:
import tarfile

In [ ]:
tarf = tarfile.open('/home/wrkshp/nexo/slurm_data/Tl208/ExternalGammas_light-v2-Tl208-5000-seed_11-20.tar.gz', 'r:gz')
print(tarf.getnames())

In [ ]:
taru(DB)

In [ ]:
file_db[-10:]

In [ ]:
a=subprocess.check_output("cat /home/wrkshp/extras/Movenotes.txt",shell=True)
print(a)

In [ ]:
a=os.path.exists("/home/wrkshp/slac_storage/sshfs/nfs/slac")
print(a)
if a ==False:
    os.system("sshfs jbane@dtn01.slac.stanford.edu:/nfs /home/wrkshp/slac_storage/sshfs/nfs")
a=os.path.exists("/home/wrkshp/slac_storage/sshfs/nfs/slac")
print(a)

os.system("/bin/fusermount -u /home/wrkshp/slac_storage/sshfs/nfs")
a=os.path.exists("/home/wrkshp/slac_storage/sshfs/nfs/slac")
print(a)


In [ ]:
a=os.path.exists("/home/wrkshp/slac_storage/sshfs/nfs/slac")
print(a)

In [ ]:
alpha=0.463
beta =0.020
gamma=1.99e-5

In [ ]:
TFileHP = uproot.open('/home/wrkshp/nexo/analysis/calib_ana/Co60-recon.root')
recon = TFileHP['reconTree']
dfHP = recon.arrays('*')
print(len(dfHP)

In [ ]:
dfHP.keys()

In [ ]:
dfHP[b'ReconZpos'][:8]

In [ ]:
dfHP[b'NumReconClusters'][:8]

In [ ]:
#def recover_recon(tgt="Co60",config="ExternalGammas_light"):
tgt="Co60"
config="ExternalGammas_light"
seed=51

print("/home/wrkshp/nEXO/slurm_data/%s/Unzipped/%s-%s-5000-seed?.tar.gz"%(tgt,config,tgt))

tarFiles=glob.glob("/home/wrkshp/nEXO/slurm_data/%s/Unzipped/%s-%s-5000-seed*.tar.gz"%(tgt,config,tgt))

zipFiles[10]
target_file="%s-%s-5000-seed%i_recon.root"%(config,tgt,seed)

#Search zipped files for possible matches.
Pos_matches=[]
for i,tarFile in enumerate(tarFiles[1:]):
    
    print(i,tarFile,"\n\n")
    tarF = tarfile.open(tarFile)
    try:
        tarlist=tarF.getnames()
    except:
        print("Issue with ",tarFile,'\n')
        continue
    
    for file in tarlist:
        if file.find("_recon") >0:
            print(file)
            tarF.extract(file,"../nEXO/slurm_data/recon/")

    

print("DDDDDDDD")
    
    

In [ ]:
Pos_matchesmatchesmatchesmatches_matches_matches_matches

In [ ]:
tarFile=tarFiles[14]



#Tarf = tarfile.open(tarFile)
#try:
#    tarlist=Tarf.getnames()
#except:
#    print("Issue with ",tarFile)

    
#tarObject_name=tarlist[0]    
#Tarf.extract(tarObject_name,"../nEXO/slurm_data/recon/")


In [ ]:
Tarf.getnames()